<a href="https://colab.research.google.com/github/kvmkrao/ksp-solvers/blob/master/iterative_scipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://docs.scipy.org/doc/scipy/reference/sparse.linalg.html
import sys
import scipy.sparse as sp
import scipy.sparse.linalg
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy import linalg
from scipy.sparse import identity
import time

iter=1 
def report(x):
  global iter; 
  print(f'iteration {iter} norm {np.linalg.norm(x)}')
  iter = iter+1; 

class gmres_counter(object):
    def __init__(self, disp=True):
        self._disp = disp
        self.niter = 0
    def __call__(self, rk=None):
        self.niter += 1
        if self._disp:
            print('iter %3i\trk = %s' % (self.niter, str(rk)))

#A = np.array([[1, 2, 0, 0], [0, 3, 4, 0], [0, 0, 5, 6], [7, 0, 8, 9]])
#A = sp.csr_matrix(A)
N = 50 
#A = np.array([    [5, 2, 1, 1],     [2, 6, 2, 1],     [1, 2, 7, 1],     [1, 1, 2, 8] ])
#A = sp.diag([[4, 2, 1, 1],     [2, 5, 2, 1],     [1, 2, 4, 1],     [1, 1, 2, 4] ], shape=[N, N], format='csc')
#b = np.array([29, 31, 26, 19]) 
initial_guess = np.ones(4)
# 4x1  + 2x2 + x3 + x4 = 29 
# 2x1  + 5x2 + 2x3 + x4 = 31 
# x1   + 2x2 + 4x3 + x4 = 26 
# x1   + x2  + 2x3 + 4x4 = 19 

A = sp.diags([1, -2, 1], [1, 0, -1], shape=[N, N], format='csc')
print(A) 
#A = sp.rand(N, N, density=0.5)
b = np.ones(N) 
D = -2 * identity(N) # np.zeros([N,N]) 
#print(D) 
invD = scipy.sparse.linalg.inv(D)
#print(invD) 

# sparse solve 
#x = sp.linalg.spsolve(A, b)
#print(np.linalg.norm(A * x - b))

# dense direct solver 
x = np.linalg.solve(A.todense(), b)
#print(x) 

#LU factorization Ax = b -> LUx = b 
lu = sp.linalg.splu(A) # sp.sparse.splu  or sp.sparse.spilu 
x = lu.solve(b)
#print(lu.L) 
#print(lu.U) 
#print(x) 

counter = gmres_counter()
print(" CG solver")
t0 = time.clock()
x, info = sp.linalg.cg(A, b,callback=report)
t1 = time.clock()
print(x)
print("wall clock time",t1-t0)

print(" Preconditioned CG solver") 
x, info = sp.linalg.cg(A, b,M=invD,callback=report)
print(x)

print("BiCG solver") 
x, info = sp.linalg.bicg(A, b,maxiter=10, callback=report)
print(x)

print("BiCGStab solver") 
x, info = sp.linalg.bicgstab(A, b,maxiter=10, callback=report)
print(x)

print("GMRES solver") 
x, info = sp.linalg.gmres(A, b,maxiter=10, callback=report)
print(x)

print("LGMRES solver") 
x, info = sp.linalg.lgmres(A, b,atol=1e-10,maxiter=10, callback=report)
print(x)

print("MINRES solver") 
x, info = sp.linalg.minres(A, b,maxiter=10, callback=report)
print(x)

print("QMR solver") 
x, info = sp.linalg.qmr(A, b,maxiter=10, callback=report)
print(x)

  (0, 0)	-2.0
  (1, 0)	1.0
  (0, 1)	1.0
  (1, 1)	-2.0
  (2, 1)	1.0
  (1, 2)	1.0
  (2, 2)	-2.0
  (3, 2)	1.0
  (2, 3)	1.0
  (3, 3)	-2.0
  (4, 3)	1.0
  (3, 4)	1.0
  (4, 4)	-2.0
  (5, 4)	1.0
  (4, 5)	1.0
  (5, 5)	-2.0
  (6, 5)	1.0
  (5, 6)	1.0
  (6, 6)	-2.0
  (7, 6)	1.0
  (6, 7)	1.0
  (7, 7)	-2.0
  (8, 7)	1.0
  (7, 8)	1.0
  (8, 8)	-2.0
  :	:
  (41, 41)	-2.0
  (42, 41)	1.0
  (41, 42)	1.0
  (42, 42)	-2.0
  (43, 42)	1.0
  (42, 43)	1.0
  (43, 43)	-2.0
  (44, 43)	1.0
  (43, 44)	1.0
  (44, 44)	-2.0
  (45, 44)	1.0
  (44, 45)	1.0
  (45, 45)	-2.0
  (46, 45)	1.0
  (45, 46)	1.0
  (46, 46)	-2.0
  (47, 46)	1.0
  (46, 47)	1.0
  (47, 47)	-2.0
  (48, 47)	1.0
  (47, 48)	1.0
  (48, 48)	-2.0
  (49, 48)	1.0
  (48, 49)	1.0
  (49, 49)	-2.0
 CG solver
iteration 1 norm 176.7766952966369
iteration 2 norm 341.3180335112694
iteration 3 norm 494.48559129665244
iteration 4 norm 636.556360426946
iteration 5 norm 767.8163843003091
iteration 6 norm 888.5617592491813
iteration 7 norm 999.0995946350894
iteration 8 norm 109

/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:208: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  'is in the CSC matrix format', SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
